## Preface

You are encouraged to experiment with the code, run it on your own data, and exchange knowledge with other participants. The purpose is to familiarise yourself with chunked pyramidal image files.

If you don't have your own data, use the result of the `convert-tiff-to-ome-zarr.ipynb`, an OME-zarr file called `chunked_pyramidal_tibia`. It is a microCT scan of a mouse shin bone. As part of this tutorial, you will read in the OME-zarr file, and threshold it in-place.

To get a visual representation of the chunks of the file, you will give each thresholded chunk a different label.

The data is relatively small (~250 MB) - to make this tutorial run fast, but in "the wild" the techniques shown here are mostly useful for large data, that doesn't fit into memory.

## Opening the zarr Group

First, we open the `zarr` Group

In [ ]:
from pathlib import Path
import zarr

path = Path("/Volumes/T7/tibiae/pyramidal_chunked_tibia")
assert path.exists()
zarr_group = zarr.open(path, mode="r+")


Next, we use `ome_zarr_models` to check the file for validity and explore it's metadata

In [ ]:
from ome_zarr_models import open_ome_zarr

ome_zarr_group = open_ome_zarr(zarr_group)
print(f"Type of the dataset: {type(ome_zarr_group)}")
print(f"OME-Zarr version: {ome_zarr_group.ome_zarr_version}")

Then, we create an OME-zarr Image from the `zarr` group.

In [ ]:
from ome_zarr_models.v04 import Image

ome_zarr_image = Image.from_zarr(zarr_group)


We can now access the various levels of the pyramid through the OME-zarr image metadata. Let's access level 0.

In [ ]:

metadata = ome_zarr_image.attributes
zarr_array = zarr_group[metadata.multiscales[0].datasets[0].path]

In [ ]:
chunk_shape = zarr_array.chunks
zarr_array_shape = zarr_array.shape

color_index=1
for i in range(0, zarr_array_shape[0], chunk_shape[0]):
    for j in range(0, zarr_array_shape[1], chunk_shape[1]):
        for k in range(0, zarr_array_shape[2], chunk_shape[2]):
            chunk = zarr_array[
                i:i+chunk_shape[0],
                j:j+chunk_shape[1],
                k:k+chunk_shape[2]
            ]
            # threshold chunk
            chunk = (chunk > 50) 

            # give chunk a new index

            chunk = chunk * color_index
            zarr_array[
                i:i+chunk_shape[0],
                j:j+chunk_shape[1],
                k:k+chunk_shape[2]    
            ] = chunk

            # update color to the next index
            # the data is unsigned int 8
            # so we repeat colours after reaching 255 chunks
            color_index = (color_index+1)%256
            if color_index == 0:
                color_index = color_index+1
            print(f"Thresholded chunk at ({i}, {j}, {k})")
            print(f"colour: {color_index}")



Finally, we have to update the levels higher up the pyramid:

In [ ]:

for d in [0,1,2]:
    level_d_array = zarr_group[ome_zarr_image.attributes.multiscales[0].datasets[d].path]
    level_d_array[:] = zarr_array[::2**d, ::2**d, ::2**d]


Finally, we can visualise the result in `napari`

In [ ]:
import napari

viewer = napari.Viewer()
viewer.open(path, plugin="napari-ome-zarr")
napari.run()

Right-click on the layer, and hit convert to Labels.